In questo notebook vengono mostrati i passaggi del nostro algoritmo di associazione secondp le tre casistiche trovate.

Input algoritmo: IDCO45 (stringa), Importo Totale (float)

Output algoritmo: una lista di tuple dove ogni tupla rappresesnta una coppia (ID_PRATICA, IMPORTO).

Nota rigurdante l'output: l'output ha un formato diverso dipendentemente dal caso in cui ci si trova. In particolare, per il caso 1 si avrà una output formato da una ed una sola coppia (ID_PRATICA, IMPORTO), quindi la struttura dell'output sarà:

`[(ID_PRATICA, IMPORTO)]`.

Nel caso 2, si avranno N coppie, quindi:

`[(ID_PRATICA_1, IMPORTO_1), (ID_PRATICA_2, IMPORTO_2), ... , (ID_PRATICA_N, IMPORTO_N)]`.



Nel caso 3, si avranno K combinazioni con dentro da 1 ad M coppie, quindi si avrà una struttura di questo tipo (in questo caso K = 4):

[

    [(ID_PRATICA_1, IMPORTO_1), (ID_PRATICA_2, IMPORTO_2)],
    [(ID_PRATICA_1, IMPORTO_3), (ID_PRATICA_2, IMPORTO_4), ... , (ID_PRATICA_M, IMPORTO_M)],
    [(ID_PRATICA_1, IMPORTO_1), (ID_PRATICA_2, IMPORTO_3), (ID_PRATICA_3, IMPORTO_4)],
    [(ID_PRATICA_1, IMPORTO_3)]
]

In [8]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

mandati_non_associati = pd.read_excel('data/03158_NON_ASSOCIATI_13_05_24.xls')
mandati_non_associati["DTINPUT"] = pd.to_datetime(mandati_non_associati["DTINPUT"], format="%d.%m.%Y")

WARNING *** file size (1029287) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
# laod dataset delle pratiche e preprocess it
pratiche_new = pd.read_csv('data/03158_pratiche_17_05_2024.csv', delimiter=";", encoding="latin1")
pratiche_new["DATA_STATO"] = pd.to_datetime(pratiche_new["DATA_STATO"], format="%d/%m/%Y")
pratiche_new['IMPORTO_RATA'] = pratiche_new['IMPORTO_RATA'].str.replace(',', '.').astype(float)

enti = pd.read_excel('data/03158_IBAN_ENTI_13_05_24.xls')

# Caso 1

Come input avremo l'IDCO45 e importo totale. Da esso è necessario estrarre l'IBAN. Dall'IBAN poi estrarremmo l'IDENTE perchè dobbiamo andare a guardare nel dataset delle pratiche e questo è indicizzabile solo dall'IDENTE.

Abbiamo già estratto dall'IDCO45 un IBAN di esempio, che è:
IT16W0338001600000014795012 e conosciamo già l'importo totale (perchè viene fornito in input) che in questo caso è 193.0.

Ora cerchiamo l'ente corrispondente all'IBAN di esempio e ne prendiamo il IDENTE.


In [10]:
enti[enti['IBAN_ACCREDITO'] == 'IT16W0338001600000014795012']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,IDENTE,IBAN_ACCREDITO,DATA_INS,FG_ATTIVO
14067,12503,IT16W0338001600000014795012,2023-06-13 18:11:46,S


Ora, se applichiamo il seguente filtro e questo restitusice una row, allora siamo per forza in un caso 1 e restituiamo in output la coppia (ID_PRATICA, IMPORTO_RATA) che sarebbe quindi: [(68149, 193.0)].

NOTA: nel caso 1 dobbiamo filtrare sempre per STATO == "Perfezionamento"

In [15]:
pratiche_new[(pratiche_new['IDENTE'] == 12503) & (pratiche_new['IMPORTO_RATA'] == 193.0) & ((pratiche_new['STATO'] == "Perfezionamento"))]

,ID_PRATICA,STATO,IMPORTO_RATA,ENTE_PAGANTE,IDENTE,DATA_STATO
26825,68149,Perfezionamento,193.0,AMAZON ITALIA TRANSPORT,12503,2020-12-30


# CASO 2

Come input avremo sempre l'IDCO45 e importo totale. Da esso è necessario estrarre l'IBAN. Dall'IBAN poi estrarremmo l'IDENTE perchè dobbiamo andare a guardare nel dataset delle pratiche e questo è indicizzabile solo dall'IDENTE.

Abbiamo già estratto dall'IDCO45 un IBAN di esempio, che è:
IT09A0538704599000044194533 e conosciamo già l'importo totale (perchè viene fornito in input) che in questo caso è 786.0.

Ora cerchiamo l'ente corrispondente all'IBAN di esempio e ne prendiamo il IDENTE.

In [13]:
enti[enti['IBAN_ACCREDITO'] == 'IT09A0538704599000044194533']

,IDENTE,IBAN_ACCREDITO,DATA_INS,FG_ATTIVO
16261,3800,IT09A0538704599000044194533,2023-10-18 09:57:08,S


Ora, come fatto nel caso 1, facciamo il filtro per IDENTE e l'importo totale. Se avrò in output una singola row, allora sarà un caso 1 come detto sopra. Se invece non ottengo nessuna row, allora ricadiamo nel caso 2 o nel caso 3.

NOTA: anche nel caso 2 dobbiamo filtrare sempre per STATO == "Perfezionamento"

In [28]:
pratiche_new[(pratiche_new['IDENTE'] == 3800) & (pratiche_new['IMPORTO_RATA'] == 786.0) & ((pratiche_new['STATO'] == "Perfezionamento"))]

,ID_PRATICA,STATO,IMPORTO_RATA,ENTE_PAGANTE,IDENTE,DATA_STATO


Nessuna row in output! Okay, quindi siamo in un caso 2 o caso 3. Andiamo a verificare se siamo in un caso 2 facendo il filtro sull'IDENTE:

In [27]:
pratiche_new[(pratiche_new['IDENTE'] == 3800) & (pratiche_new['STATO'] == "Perfezionamento")]

,ID_PRATICA,STATO,IMPORTO_RATA,ENTE_PAGANTE,IDENTE,DATA_STATO
15332,115947,Perfezionamento,272.0,ATERP CALABRIA - AZIENDA TERRITORIALE EDILIZIA PUBBLICA,3800,2023-11-16
15334,84306,Perfezionamento,220.0,ATERP CALABRIA - AZIENDA TERRITORIALE EDILIZIA PUBBLICA,3800,2022-01-04
24656,122490,Perfezionamento,294.0,ATERP CALABRIA - AZIENDA TERRITORIALE EDILIZIA PUBBLICA,3800,2024-01-31


Ottengo N rows. Siamo nel caso 2 se e solo se la somma dei singoli IMPORTO_RATA è uguale all'importo totale in input (che era 786.0) filtrando per STATO == "Perfezionamento", andiamo quindi a fare questa somma:

In [14]:
pratiche_new[(pratiche_new['STATO'] == "Perfezionamento") & (pratiche_new['IDENTE'] == 3800)]['IMPORTO_RATA'].sum()

786.0

La somma è esattamente 786.0! Quindi ricadiamo nel caso 2 e restituiamo in output le 3 coppie pratiche importo:
[(115947, 272.0), (84306, 220.0), (122490, 294.0)]

# CASO 3

Se la somma dei singoli importi non corrisponde all'importo totale, allora siamo nel caso 3 e qui viene runnato l'algoritmo di backtracking per la ricerca delle combinazioni.

Input: IBAN: IT15G0538778590000044312451 con Import Totale: 2334.0

In [16]:
enti[enti['IBAN_ACCREDITO'] == 'IT15G0538778590000044312451']

,IDENTE,IBAN_ACCREDITO,DATA_INS,FG_ATTIVO
14161,20554,IT15G0538778590000044312451,2023-06-16 16:03:33,S


In [17]:
pratiche_new[(pratiche_new['IDENTE'] == 20554) & (pratiche_new['IMPORTO_RATA'] == 2334.0) & ((pratiche_new['STATO'] == "Perfezionamento"))]

,ID_PRATICA,STATO,IMPORTO_RATA,ENTE_PAGANTE,IDENTE,DATA_STATO


In [18]:
pratiche_new[(pratiche_new['IDENTE'] == 20554) & ((pratiche_new['STATO'] == "Perfezionamento"))]

,ID_PRATICA,STATO,IMPORTO_RATA,ENTE_PAGANTE,IDENTE,DATA_STATO
79602,97777,Perfezionamento,366.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79603,97483,Perfezionamento,250.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79604,97176,Perfezionamento,348.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79605,96870,Perfezionamento,270.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79607,96077,Perfezionamento,330.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79608,95607,Perfezionamento,337.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79611,93848,Perfezionamento,200.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79612,93674,Perfezionamento,233.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2022-05-07
79614,80118,Perfezionamento,260.0,FONDAZIONE DI RELIGIONE E DI CULTO CASA SOLLIEVO DELLA SOFFERENZA - OPERA DI SAN PIO DA PIETRELCINA,20554,2021-11-08


In [19]:
pratiche_new[(pratiche_new['STATO'] == "Perfezionamento") & (pratiche_new['IDENTE'] == 20554)]['IMPORTO_RATA'].sum()

2594.0

Essendo che questa somma è diversa dall'importo totale che abbiamo in input (che è 2334.0), allora ricadiamo nel caso 3 dove dobbiamo andare a cercare le combinazioni tramite l'algoritmo di backtracking. Quindi viene lanciata la funzione:

```python

k_combinazioni_restituite_caso3 = get_combinations()

```

Quindi in questo caso verranno restituite in output le combinazioni trovate, di cui al massimo 1 sarà corretta. Se nessuna è corretta (nota: sarà il dipendente a deciderlo se è corretta o meno), allora questo mandato verrà mandato in stato di Sinistro. 

NOTA riguardante l'output restituito: quando vengono incontrati nei casi edge, non viene restituta nessuna coppia / liste di coppie, ma per il mommento restituiamo None. Nella fattispecie, nella funzione `get_association()` vengono gestiti questi casi None:
```python
if iban == "IT75I0315801600BO0990000300":
    self.iban_banca_sistema += 1
    return None, "banca_sistema"

if "INPS" in idco45 or "I.N.P.S" in idco45:
    self.iban_inps += 1

if not iban:
    self.iban_none += 1
    return None, "iban_none"

id_ente = self.get_id_ente(iban, iban_to_idente)
if not id_ente:
    return None, "identi_none"
```

IBAN Giuseppe: IT75I0315801600BO0990000300
ID ENTE: 123
Rata di Giuseppe: 200€

IBAN FSannicola: IT75I03158016075975949567
ID ENTE: 123
Rata di FSannicola: 200€

IBAN Giacomo: IT58JFOGJAOIGFEOIGFIOFJIJFIJ
ID ENTE: 123
Rata di Giacomo: 200€